In [1]:
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
import os

file_path = 'C:/Users/user/Downloads'  # 다운로드 받을 곳. 이 폴더는 비워놓고 시작하는게 좋음.
driver = webdriver.Chrome() # 괄호 안에 '크롬 웹 드라이버 위치' 넣어주기

In [2]:
# 크롬 웹 드라이버는 자기 크롬 버전에 맞는 것 다운 받기. 크롬 -> 설정 -> 메뉴 맨 밑 크롬 정보 에서 버전 확인 가능
# 괄호 안에 위치 넣어 줄 때 (드라이버 속해있는 폴더 경로)/chromedriver.exe 까지 넣기
driver.implicitly_wait(5)

# 관광지별 대시보드 페이지
driver.get('https://datalab.visitkorea.or.kr/datalab/portal/loc/getTourDataForm.do')

# 로그인
sleep(0.5)
driver.find_element_by_css_selector('#wrap > header > div.util-wrap > div > div.member > a.btn_login.border-right').click()
sleep(0.5)
driver.find_element_by_css_selector('#mbrId').send_keys('wnstjq1254@naver.com')
sleep(0.5)
driver.find_element_by_css_selector('#mbrPw').send_keys('wnstjq12!')
sleep(0.5)
driver.find_element_by_css_selector('#content > div > div > input.table-btn.floatright.btn').click()

sleep(0.5)
driver.find_element_by_css_selector('#listView').click()  # 목록으로 보기 버튼

In [3]:
res = {}
for times in range(25):  # 장에 대한 반복문(10페이지씩 10장)
    sleep(1.5)
    for i in range(10):  # 페이지에 대한 반복문(10페이지 보고 다음 10페이지 반복)
        sleep(1.5)
        for j in range(10):  # 목록에 띄워진 10개의 여행지에 대한 반복문
            # 목록에 띄워진 여행지들 순서대로 클릭
            dest = driver.find_element_by_css_selector(f'#tabCon2 > ul > li:nth-child({j+1}) > div:nth-child(1) > a')
            id = dest.get_attribute("href").split('=')[-1]
            dest = dest.get_attribute("innerText")
            res.setdefault(dest, 0)
            res[dest] = id
        if i != 9:
            # 다음 페이지로 옮기기
            driver.find_element_by_css_selector(f'#tabCon2 > div.paging.mt55 > a:nth-child({i+4})').click()
        elif i == 9:
            # 만약 10페이지라면 다음 장으로 옮기기
            driver.find_element_by_css_selector('#tabCon2 > div.paging.mt55 > a.table-forward').click()
    res, len(res)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#tabCon2 > ul > li:nth-child(8) > div:nth-child(1) > a"}
  (Session info: chrome=107.0.5304.107)


In [16]:
import pandas as pd
df = pd.DataFrame(res, index=[0]).T.reset_index(drop=False)
df.columns = ("destination", "id")
df

,destination,id
0,덕포해수욕장,127939
1,백천사(사천),127796
2,달실마을,127035
3,설악워터피아,126714
4,이호테우해변,126448
...,...,...
2492,연미사(안동),337501
2493,광덕사,298084
2494,익산교도소세트장,2594360
2495,양양지(위양못),2545984


In [17]:
df.to_csv("output/dest_id.csv", index=False)